# Testes Estatísticos em Séries Dessazonalizadas

Este notebook aplica **testes estatísticos** às séries temporais de **mortalidade** e **morbidade cardiovascular** já dessazonalizadas.  
O objetivo é identificar **mudanças significativas** nos padrões **antes e depois da pandemia de COVID-19**.

## Etapas:

- Carregamento dos dados dessazonalizados;
- Aplicação de testes estatísticos para comparação dos períodos;
- Armazenamento dos resultados para análises posteriores.

In [1]:
# Imports básicos
import pandas as pd
from pathlib import Path


## Importação de Funções do Projeto

Importação dos módulos desenvolvidos no projeto:

- `carregamento.py` – leitura dos dados tratados;
- `stl.py` – dessazonalização com STL;
- `estatisticas.py` – aplicação dos testes estatísticos.


In [2]:
# Importa funções personalizadas do projeto
from scripts.carregamento import carregar_dados_tratados
from scripts.dessazonalizacao import dessazonalizar_em_lote
from scripts.testes_estatisticos import aplicar_testes_estatisticos

## Carregamento e Dessazonalização dos Dados

- Os dados previamente tratados são carregados e organizados;
- Cada série de mortalidade e morbidade é submetida à dessazonalização com STL;
- Isso garante que os testes avaliem apenas variações associadas a tendências ou instabilidades, sem interferência de padrões sazonais.

In [3]:
# Carrega os dados tratados
dados = carregar_dados_tratados()

# Aplica dessazonalização nas séries de mortalidade e morbidade
dados_dessaz = {
    "mortalidade": dessazonalizar_em_lote(dados["mortalidade"]),
    "morbidade": dessazonalizar_em_lote(dados["morbidade"])
}


## Aplicação dos Testes Estatísticos

Para cada evento cardiovascular (óbito ou internação), são aplicados testes estatísticos comparando o comportamento da série nos períodos:

- Pré-pandemia  
- Durante/pós-pandemia

Os testes aplicados incluem:

- **Teste de Mann-Whitney** – avalia se houve mudança significativa na mediana dos valores entre os períodos, permitindo concluir se há evidência de aumento (ou redução) nos eventos analisados, mesmo em distribuições não normais;

- **Teste de Levene** – verifica se houve alteração na variabilidade (dispersão) dos dados, indicando instabilidade no sistema de saúde;

- **Rank Biserial com Intervalo de Confiança (IC)** – mede o tamanho do efeito entre os períodos, ou seja, o quão consistentemente os valores do pós-pandemia são maiores (ou menores) que os do pré. Além disso, o intervalo de confiança permite avaliar a **robustez** do efeito observado: se o IC exclui o zero, isso indica que a diferença entre os períodos é consistente e provavelmente não ocorreu ao acaso.

Os resultados são armazenados em listas para posterior análise consolidada.


In [4]:
# Testes estatísticos para mortalidade dessazonalizada
resultados_mortalidade_dessaz = []
for evento, df in dados_dessaz["mortalidade"].items():
    res = aplicar_testes_estatisticos(df, coluna_valor="dessazonalizado")
    res["evento"] = evento
    resultados_mortalidade_dessaz.append(res)

# Testes estatísticos para morbidade dessazonalizada
resultados_morbidade_dessaz = []
for evento, df in dados_dessaz["morbidade"].items():
    res = aplicar_testes_estatisticos(df, coluna_valor="dessazonalizado")
    res["evento"] = evento
    resultados_morbidade_dessaz.append(res)

## Organização e Salvamento dos Resultados

- Os resultados dos testes estatísticos são organizados em um `DataFrame`;
- Esse `DataFrame` é exportado em formato `.csv`;
- O arquivo final é salvo no diretório `/resultados`, onde poderá ser consultado ou utilizado em análises e visualizações futuras.


In [5]:
# Converte os resultados para DataFrame
df_mortalidade_dessaz = pd.DataFrame(resultados_mortalidade_dessaz)
df_morbidade_dessaz = pd.DataFrame(resultados_morbidade_dessaz)

# Salva resultados em /resultados
pasta_resultados = Path("..") / "resultados"
pasta_resultados.mkdir(exist_ok=True)

df_mortalidade_dessaz.to_csv(pasta_resultados / "testes_mortalidade_dessazonalizado.csv", index=False)
df_morbidade_dessaz.to_csv(pasta_resultados / "testes_morbidade_dessazonalizado.csv", index=False)

# head para verificar os dados
df_mortalidade_dessaz.head()


,mannwhitney_p,rank_biserial_r,rank_biserial_ic,levene_p,media_pre,media_pos,cv_pre,cv_pos,evento
0,0.999783,-0.510417,"(-0.720529513888889, -0.28815104166666666)",0.133322,905.190909,972.917287,0.059540,0.085833,obitos_avc
1,0.910504,-0.194444,"(-0.44969618055555555, 0.06948784722222219)",0.010407,7805.927072,7860.845306,0.026718,0.053373,obitos_infarto
2,1.000000,-0.869792,"(-0.9774739583333334, -0.7256944444444444)",0.000129,2209.241409,2547.811267,0.021958,0.088857,obitos_insuf_cardiaca
3,0.999997,-0.652778,"(-0.8663628472222222, -0.40620659722222224)",0.447212,9.289353,12.412020,0.443882,0.212498,obitos_miocardite
4,0.999982,-0.598958,"(-0.8385416666666666, -0.3350694444444444)",0.631846,126.679205,146.947576,0.172203,0.119818,obitos_trombose


### Tabela-resumo: Significância estatística e intervalo de confiança do rank biserial

| Evento                  | p (Mann-Whitney) | Significância | IC 95% (r_rb)                         | IC exclui 0 |
|------------------------|------------------|---------------|--------------------------------------|-------------|
| Óbitos por AVC         | 0.9998           | ✗             | (−0.72, −0.29)                        | ✓           |
| Óbitos por Infarto     | 0.9105           | ✗             | (−0.45, +0.07)                        | ✗           |
| Insuficiência Cardíaca | 1.0000           | ✗             | (−0.98, −0.73)                        | ✓           |
| Miocardite             | 1.0000           | ✗             | (−0.87, −0.41)                        | ✓           |
| Trombose               | 0.9999           | ✗             | (−0.84, −0.34)                        | ✓           |


### **Comentários dos resultados:**

Apesar de a maioria dos testes de Mann-Whitney não terem apresentado significância estatística (p > 0.05), a análise dos efeitos práticos — medidos pelo **rank biserial com intervalo de confiança (IC)** — revela **sinais consistentes de agravamento** nos eventos cardiovasculares após o início da pandemia.

Nos 5 eventos de mortalidade analisados até aqui:

- **4 apresentaram intervalos de confiança que excluem o zero**, indicando efeitos práticos robustos mesmo sem significância clássica.
- Apenas **1 evento (infarto)** apresentou intervalo que inclui o zero, tornando o efeito observado incerto.
- Os efeitos observados nesses 4 eventos foram todos **negativos e de magnitude grande**, sugerindo aumento sistemático nos valores pós-pandemia.

O *rank biserial* indica **a direção e a força da diferença** entre os períodos, enquanto o IC fornece **a confiabilidade** dessa diferença. Quando ambos apontam para um efeito consistente, mesmo sem p < 0.05, temos **evidência prática relevante**.

Esses achados reforçam a importância de se utilizar medidas de efeito com IC — e não apenas testes de hipótese — especialmente em contextos com variabilidade elevada ou eventos raros, como observado em miocardite.
